# Dicom modification 

In [ ]:
radiologists = ['Andres', 'Daniel']
experiment_keyword = 'MASK_DELINEATION'

selected_patients = [
    'ACV-185',
    'ACV-187',
    'ACV-191',
    'ACV-192',
    'ACV-193',
    'ACV-196',
    'ACV-202',
    'ACV-204',
    'ACV-205',
    'ACV-228',
    'ACV-232',
    ]

In [ ]:
from pydicom.fileset import FileSet
import numpy as np
import os
from tqdm import tqdm
import pydicom
import pydicom_seg
import subprocess
import SimpleITK as sitk
import nibabel as nib
from copy import deepcopy
import logging
from glob import glob
from pathlib import Path

data_path = Path('/data/Datasets/stroke/Foscal_uploaded_data/ZIP DICOM/')
dump_dir = Path('/data/sangohe_pupils/srazul/dicomdump')
radiologists = ['Andres', 'Daniel']


def get_image_id(path, patient_id):
    """ Gets an id from a folder full of files formated as follow
    <prefix><id><suffix>
    where id is a number of 6 digits
    """
    prefix = str(patient_id) + '-'
    suffix = '.dcm'
    # TODO: this has to be fixed since is relies on a hardcoded value
    id_list = [int(p.split('-')[-1].replace(suffix,  '')) for p in  os.listdir(path)]
    if not id_list:
        cur_id = '{0:06d}'.format(0)
    else: 
        last_id = max(id_list)
        cur_id = '{0:06d}'.format(last_id + 1)
    
    return f'{prefix}{cur_id}{suffix}'

   
def main():
    """ Changes the name of the patients for the name of a doctor an the
    number of the patient and saves the resultant dicoms in a folder
    """

    for rad in radiologists:
        output_dir = dump_dir / rad
        output_dir.mkdir(exist_ok=True)
        for patient_id in tqdm([p for p in os.listdir(data_path) if p in selected_patients], f'rad: {rad}'):
            pat_path = data_path / patient_id
            dicomdir_list = glob(f'{pat_path}/**/DICOMDIR')
            for dicomdir in dicomdir_list:
                check = FileSet(dicomdir)
                for instance in check:
                    temp = instance.load()
                    #pname = str(instance.PatientName).replace('ACV', rad)
                    temp.PatientID = rad
                    temp.StudyDescription = experiment_keyword
                    modality = temp.modality.replace(' ', '_')
                    img_output_file = Path(os.path.join(output_dir, modality, get_image_id(output_dir, patient_id)))
                    img_output_file.mkdir(exist_ok = True)
                    temp.save_as(img_output_file)
                    return 0
main()